In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import os, sys
from os import listdir
from os.path import isfile, join
from pathlib import Path
from glob import glob


from google.colab.patches import cv2_imshow

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input

from PIL import Image
import cv2


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/ozon_data_img/dir.zip'


In [ ]:
data_path = Path('dir/test')
class_names = [f for f in listdir(data_path) if isdir(join(data_path, f))]

In [ ]:
base_dir = Path('/content/dir')
folders = []
test_dir = base_dir / 'test/'
train_dir = base_dir / 'train/'

In [ ]:
class Data():
  def __init__(self):
    pass
  
  def prepare_data(preprocess_input, BATCH_SIZE=512, IMG_SHAPE=150, val_dir=test_dir, train_dir=train_dir, class_mode='binary'):

    image_gen_train = ImageDataGenerator(preprocessing_function=preprocess_input)
    image_gen_val = ImageDataGenerator(preprocessing_function=preprocess_input, fill_mode='nearest')

    train_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,
                                                        directory=train_dir,
                                                        shuffle=True,
                                                        target_size=(IMG_SHAPE,IMG_SHAPE),
                                                        class_mode=class_mode)

    val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                                  directory=val_dir,
                                                                  shuffle=False,
                                                                  target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                                  class_mode=class_mode)
    return train_data_gen, val_data_gen

In [ ]:
data = Data()
train_data_gen, val_data_gen = Data.prepare_data(preprocess_input=preprocess_input, train_dir=train_dir, val_dir=test_dir)

In [ ]:
class Classifier:
  def __init__(self):
    self.IMG_SHAPE = (150, 150, 3)
    # базовая модель -- EfficientNetV2M
    self.base_model = tf.keras.applications.EfficientNetV2M(input_shape=self.IMG_SHAPE, weights='imagenet', include_top=False)  #
    self.base_model.trainable = False # замораживаем всю базовую модель

  def unfreeze(self):
    self.base_model.trainable = True
    print("Количество слоев в базовой модели: ", len(self.base_model.layers), '\n')
    fine_tune_at = 600
    # все слои до -- заморозим
    for layer in self.base_model.layers[:fine_tune_at]:
      layer.trainable =  False


  def extra_layers(self,metrics='accuracy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), num_classes=None, loss=None): #tf.keras.optimizers.Adam(learning_rate=0.001)
    if num_classes == 2:
      self.model = tf.keras.Sequential([
      self.base_model,
      # tf.keras.layers.Dropout(0.5),
      # tf.keras.layers.BatchNormalization(),
      # tf.keras.layers.Dense(100, activation='elu'),
      # tf.keras.layers.Dropout(0.5),
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dense(1, activation='sigmoid')
      ])
    else:
      self.model = tf.keras.Sequential([
      self.base_model,
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dense(num_classes, activation='softmax')
      ])
    self.model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

    print('!the model was built with additional layers!\n')
  
  
  def fit_train(self, epochs=10, train_data_gen=train_data_gen, val_data_gen=val_data_gen):
    сheckpoint = callbacks.ModelCheckpoint('/content/efficentnetV2M-ozon.hdf5', monitor='val_acc', save_best_only=True)
    self.hist = self.model.fit_generator(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen,
    callbacks=[сheckpoint]
    )


  def fit_vizualization(self):
    plt.plot(self.hist.history['loss'])
    plt.plot(self.hist.history['val_loss'])


  def predict_classes(self, datagen=val_data_gen):
    sample_validation_images, sample_validation_labels = next(datagen)
    self.predictions = (self.model.predict(sample_validation_images) > 0.5).astype("int32").flatten()
    self.sample_validation_images = sample_validation_images
    self.sample_validation_labels = sample_validation_labels


  def show_predictions(self, images, labels, predicted_labels=None):
    # names = {0: "No", 1: "Yes"}
    # names = zip(range(74), class_names)
    plt.figure(figsize=(10,10))
    for i in range(len(predicted_labels)):
        plt.subplot(4,4, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow((images[i] + 1)/2., cmap=plt.cm.gray)
        if predicted_labels is not None:
            title_obj = plt.title(f"Real: {names[labels[i]]}. Pred: {names[predicted_labels[i]]}")
            if labels[i] != predicted_labels[i]:
                plt.setp(title_obj, color='r')
        else:
            plt.title(f"Real label: {names[labels[i]]}")

In [ ]:
%%time
ozon = Classifier()
ozon.extra_layers(num_classes=73, loss=tf.keras.losses.SparseCategoricalCrossentropy())
ozon.unfreeze()
ozon.fit_train(epochs=30)      #change valid


In [ ]:
ozon.fit_vizualization()
ozon.predict_classes(train_data_gen)    #change valid
ozon.show_predictions(ozon.sample_validation_images, ozon.sample_validation_labels, ozon.predictions)

In [ ]:
plt.plot(ozon.hist.history['accuracy'])
plt.plot(ozon.hist.history['val_accuracy'])